In [50]:
from selenium import webdriver
import numpy as np
import glob
import os
import time
import datetime
import pandas as pd
import re
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import tqdm
import parser
%matplotlib inline

### General information : BitInfoCharts

In [83]:
df4 = parser.parse_bitinfocharts()

In [85]:
df4.head(10)

,Columns,BTC,ETH,LTC
0,,Bitcoin,Ethereum,Litecoin
1,Total,16696011,96030202,54053583
2,Price,10709.8,483.15,98.34
3,Market Capitalization,1.78812e+11,4.6397e+10,5.31561e+09
4,Transactions last 24h,405002,652664,53338
5,Transactions avg. per hour,"16,875","27,194","2,222"
6,Sent last 24h,3.3466e+06,2.89955e+07,1.88351e+07
7,Sent avg. per hour,139442,1.20814e+06,784796
8,Avg. Transaction Value,8.26,44.43,353.13
9,Median Transaction Value,0.094,0.049,4.7


### BTC.com

In [81]:
df3 = parser.parse_btc(1)

In [82]:
df3.head()

,Height,Tx Count,Stripped Size(B),Size(B),Weight,Avg Fee Per Tx,Reward,Time,RewardFee
0,496722.0,2337.0,979246.0,1055378.0,3993116.0,0.000532,12.5 + 2.12582327 BTC,2017-11-29 18:38:07,2.125823
1,496721.0,2176.0,961790.0,1107704.0,3993074.0,0.000421,12.5 + 1.67932363 BTC,2017-11-29 18:23:19,1.679324
2,496720.0,2503.0,973606.0,1072358.0,3993176.0,0.000488,12.5 + 1.94703215 BTC,2017-11-29 18:19:57,1.947032
3,496719.0,2310.0,980779.0,1050334.0,3992671.0,0.000437,12.5 + 1.74473501 BTC,2017-11-29 18:11:07,1.744735
4,496718.0,2726.0,959167.0,1115480.0,3992981.0,0.000441,12.5 + 1.76145309 BTC,2017-11-29 18:06:21,1.761453


### EtherScan

In [77]:
imp.reload(parser)
df = parser.parse_ether(100)

In [78]:
df.head()

,Height,Age,txn,Uncles,Miner,GasUsed,GasLimit,Avg.GasPrice,Reward
0,4645181,Nov-29-2017 05:40:30 PM,24.0,0,Ethermine,522512 (7.77%),6722271,11.12,3.00581
1,4645180,Nov-29-2017 05:40:26 PM,179.0,0,f2pool_2,6723561 (99.92%),6728805,18.76,3.12614
2,4645179,Nov-29-2017 05:40:09 PM,173.0,0,f2pool_2,6709312 (99.81%),6722242,16.43,3.11021
3,4645178,Nov-29-2017 05:39:46 PM,98.0,0,Ethermine,6675871 (99.41%),6715685,9.58,3.06393
4,4645177,Nov-29-2017 05:39:38 PM,149.0,0,bw.com,6704478 (99.74%),6722215,18.65,3.12504


### LTC/DOGE/DASH

In [79]:
df2 = pd.concat([
    parser.parse_blockcypher('ltc'),
    parser.parse_blockcypher('doge'),
    parser.parse_blockcypher('dash')
])
df2

,block_number,blockchain,fees_satoshi,nonce,time,transactions
0,1322220,LTC.main,13210795,0,2017-11-29T17:36:57Z,135
1,1322219,LTC.main,3843798,0,2017-11-29T17:33:40Z,27
2,1322218,LTC.main,9889764,0,2017-11-29T17:33:17Z,23
3,1322217,LTC.main,1531860,0,2017-11-29T17:32:50Z,17
4,1322216,LTC.main,2395700,0,2017-11-29T17:32:36Z,16
0,1990081,DOGE.main,900000000,0,2017-11-29T17:33:42Z,10
1,1990080,DOGE.main,0,0,2017-11-29T17:33:17Z,1
2,1990079,DOGE.main,350000000,0,2017-11-29T17:32:50Z,5
3,1990078,DOGE.main,500000000,0,2017-11-29T17:32:35Z,4
4,1990077,DOGE.main,1422633524,0,2017-11-29T17:32:26Z,11
